In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
!pip install transformers

In [ ]:
import os

In [ ]:
from google.colab import files
uploaded = files.upload()
from google.colab import files
uploaded = files.upload()
from google.colab import files
uploaded = files.upload()

Saving fake reviews dataset(訓練).csv to fake reviews dataset(訓練).csv


Saving label0.csv to label0.csv


Saving label1.csv to label1.csv


In [ ]:
import pandas as pd

# 載入資料集到 pandas 的 dataframe 中
df=pd.read_csv("fake reviews dataset(訓練).csv")
df
df0=pd.read_csv("label0.csv")
df0
df1=pd.read_csv("label1.csv")
df1

#shape可以回傳目前dataframe的列數,欄數
#shape[0]回傳列數 shape[1]回傳欄數
#print(df.shape)

# 列印資料集的記錄數
print('Number of training sentences: {:,}\n'.format(df.shape[0]))



Number of training sentences: 12,303



In [ ]:
filt = (df["label"]  == 1)
print(df.loc[filt])
print('Number of training sentences: {:,}\n'.format(filt.shape[0]))

                    category  rating  label  \
2185                 Books_5       4      1   
2186                 Books_5       5      1   
2187                 Books_5       5      1   
2188                 Books_5       5      1   
2189                 Books_5       4      1   
...                      ...     ...    ...   
12298  Sports_and_Outdoors_5       5      1   
12299  Sports_and_Outdoors_5       5      1   
12300  Sports_and_Outdoors_5       5      1   
12301  Sports_and_Outdoors_5       3      1   
12302  Sports_and_Outdoors_5       4      1   

                                                   text_  
2185   It's witty and intriguing. I hang on to every ...  
2186   Good read. I really enjoyed the characters in ...  
2187   It's Dr. Seuss, how can you miss??? Great qual...  
2188   Great read.  Makes you wish you lived in Nantu...  
2189   A good read which keeps you wondering until th...  
...                                                  ...  
12298  These ice packs

In [ ]:
from transformers import BertTokenizer

# 載入 BERT 分詞器
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# 構建 text_和 labels 列表
text_= df.text_.values
labels = df.label.values #0為CG(寫手評論) 1為OR(原始評論)
print(' Original: ', text_[1])# 輸出原始句子
print('Tokenized: ', tokenizer.tokenize(text_[1]))# 將分詞後的內容輸出
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_[1])))# 將每個詞對映到詞典下標

Loading BERT tokenizer...


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

 Original:  Good book and exactly as described. The characters were well developed and believable. The writing was just as
Tokenized:  ['good', 'book', 'and', 'exactly', 'as', 'described', '.', 'the', 'characters', 'were', 'well', 'developed', 'and', 'bel', '##ie', '##vable', '.', 'the', 'writing', 'was', 'just', 'as']
Token IDs:  [2204, 2338, 1998, 3599, 2004, 2649, 1012, 1996, 3494, 2020, 2092, 2764, 1998, 19337, 2666, 12423, 1012, 1996, 3015, 2001, 2074, 2004]


In [ ]:
input_ids = []
attention_masks = []

for sent in text_:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # 輸入文字
                        add_special_tokens = True, # 新增 '[CLS]' 和 '[SEP]'
                        max_length =310,           # 填充 & 截斷長度
                        pad_to_max_length = True,
                        return_attention_mask = True,   # 返回 attn. masks.
                        return_tensors = 'pt',     # 返回 pytorch tensors 格式的資料
                   )

    # 將編碼後的文字加入到列表
    input_ids.append(encoded_dict['input_ids'])

    # 將文字的 attention mask 也加入到 attention_masks 列表
    attention_masks.append(encoded_dict['attention_mask'])

# 將列表轉換為 tensor
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# 輸出第 1 行文字的原始和編碼後的資訊
print('Original: ', text_[0])
print('Token IDs:', input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  I'm hooked on this writer and will be reading more of her work.
Token IDs: tensor([  101,  1045,  1005,  1049, 13322,  2006,  2023,  3213,  1998,  2097,
         2022,  3752,  2062,  1997,  2014,  2147,  1012,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,

In [ ]:
from torch.utils.data import TensorDataset, random_split

# 將輸入資料合併為 TensorDataset 物件
dataset = TensorDataset(input_ids, attention_masks, labels)

# 計算訓練集和驗證集大小
train_size = int(0.7 * len(dataset))
val_size = len(dataset) - train_size

# 按照資料大小隨機拆分訓練集和測試集
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

8,612 training samples
3,691 validation samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# 在 fine-tune 的訓練中，BERT 作者建議小批量大小設為 16 或 32
batch_size = 32

# 為訓練和驗證集建立 Dataloader，對訓練樣本隨機洗牌
train_dataloader = DataLoader(
            train_dataset,  # 訓練樣本
            sampler = RandomSampler(train_dataset), # 隨機小批量
            batch_size = batch_size # 以小批量進行訓練
        )

# 驗證集不需要隨機化，這裡順序讀取就好
validation_dataloader = DataLoader(
            val_dataset, # 驗證樣本
            sampler = SequentialSampler(val_dataset), # 順序選取小批量
            batch_size = batch_size
        )



for t_batch in validation_dataloader:
  print(t_batch[0])
  print(t_batch[1])
  print(t_batch[2])


tensor([[ 101, 2224, 2122,  ...,    0,    0,    0],
        [ 101, 1996, 8476,  ...,    0,    0,    0],
        [ 101, 2018, 2192,  ...,    0,    0,    0],
        ...,
        [ 101, 2023, 2338,  ...,    0,    0,    0],
        [ 101, 2307, 4031,  ...,    0,    0,    0],
        [ 101, 1045, 2001,  ...,    0,    0,    0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0,
        1, 1, 0, 1, 1, 0, 0, 1])
tensor([[ 101, 2000, 2360,  ...,    0,    0,    0],
        [ 101, 2023, 2003,  ...,    0,    0,    0],
        [ 101, 3175, 6021,  ...,    0,    0,    0],
        ...,
        [ 101, 1045, 2228,  ...,    0,    0,    0],
        [ 101, 1996, 3166,  ...,    0,    0,    0],
        [ 101, 2200, 7537,  ...,    0,    0,    0]])
tensor([[1, 1, 1,

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# 載入 BertForSequenceClassification, 預訓練 BERT 模型 + 頂層的線性分類層
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # 小寫的 12 層預訓練模型
    num_labels = 2, # 分類數 --2 表示二分類
                    # 你可以改變這個數字，用於多分類任務
    output_attentions = False, # 模型是否返回 attentions weights.
    output_hidden_states = False, # 模型是否返回所有隱層狀態.
)

# 在 gpu 中執行該模型
model.cuda()

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# 將所有模型引數轉換為一個列表
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [ ]:
# 我認為 'W' 代表 '權重衰減修復"
optimizer = AdamW(model.parameters(),
                  lr = 4e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                )


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from transformers import get_linear_schedule_with_warmup

# 訓練 epochs。 BERT 作者建議在 2 和 4 之間，設大了容易過擬合
epochs = 2
# 總的訓練樣本數
total_steps = len(train_dataloader) * epochs

# 建立學習率排程器
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np

# 根據預測結果和標籤資料來計算準確率
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    #print("pred_flat",pred_flat)
    labels_flat = labels.flatten()
    #print("labels_flat",labels_flat)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # 四捨五入到最近的秒
    elapsed_rounded = int(round((elapsed)))

    # 格式化為 hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import random
import numpy as np

# 以下訓練程式碼是基於 `run_glue.py` 指令碼:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# 設定隨機種子值，以確保輸出是確定的
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 儲存訓練和評估的 loss、準確率、訓練時長等統計指標,
training_stats = []

# 統計整個訓練時長
total_t0 = time.time()

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================


    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 統計單次 epoch 的訓練時間
    t0 = time.time()

    # 重置每次 epoch 的訓練總 loss
    total_train_loss = 0

    # 將模型設定為訓練模式。這裡並不是呼叫訓練介面的意思
    # dropout、batchnorm 層在訓練和測試模式下的表現是不同的 (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # 訓練集小批量迭代
    for step, batch in enumerate(train_dataloader):

        # 每經過40次迭代，就輸出進度資訊
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 準備輸入資料，並將其拷貝到 gpu 中
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # 每次計算梯度前，都需要將梯度清 0，因為 pytorch 的梯度是累加的
        model.zero_grad()

        # 前向傳播
        # 文件參見:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # 該函式會根據不同的引數，會返回不同的值。 本例中, 會返回 loss 和 logits -- 模型的預測結果
        loss, logits = model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels,
                             return_dict =False)

        # 累加 loss
        total_train_loss += loss.item()

        # 反向傳播
        loss.backward()

        # 梯度裁剪，避免出現梯度爆炸情況
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 更新引數
        optimizer.step()

        # 更新學習率
        scheduler.step()

    # 平均訓練誤差
    avg_train_loss = total_train_loss / len(train_dataloader)

    # 單次 epoch 的訓練時長
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    # ========================================
    #               Validation
    # ========================================
    # 完成一次 epoch 訓練後，就對該模型的效能進行驗證

    print("")
    print("Running Validation...")

    t0 = time.time()

    # 設定模型為評估模式
    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        # 將輸入資料載入到 gpu 中
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # 評估的時候不需要更新引數、計算梯度
        with torch.no_grad():
            (loss, logits) = model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels,
                                   return_dict =False)

        # 累加 loss
        total_eval_loss += loss.item()

        # 將預測結果和 labels 載入到 cpu 中計算
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # 計算準確率
        total_eval_accuracy += flat_accuracy(logits, label_ids)


    # 列印本次 epoch 的準確率
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # 統計本次 epoch 的 loss
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    # 統計本次評估的時長
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # 記錄本次 epoch 的所有統計資訊
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 2 ========
Training...
  Batch    40  of    270.    Elapsed: 0:01:02.
  Batch    80  of    270.    Elapsed: 0:02:02.
  Batch   120  of    270.    Elapsed: 0:03:05.
  Batch   160  of    270.    Elapsed: 0:04:09.
  Batch   200  of    270.    Elapsed: 0:05:13.
  Batch   240  of    270.    Elapsed: 0:06:18.

  Average training loss: 0.17
  Training epcoh took: 0:07:05

Running Validation...
  Accuracy: 0.93
  Validation Loss: 0.19
  Validation took: 0:01:12

======== Epoch 2 / 2 ========
Training...
  Batch    40  of    270.    Elapsed: 0:01:06.
  Batch    80  of    270.    Elapsed: 0:02:11.
  Batch   120  of    270.    Elapsed: 0:03:16.
  Batch   160  of    270.    Elapsed: 0:04:21.
  Batch   200  of    270.    Elapsed: 0:05:27.
  Batch   240  of    270.    Elapsed: 0:06:32.

  Average training loss: 0.04
  Training epcoh took: 0:07:20

Running Validation...
  Accuracy: 0.96
  Validation Loss: 0.15
  Validation took: 0:01:12

Training complete!
Total training took 0:16

In [ ]:
import pandas as pd

# 保留 2 位小數
pd.set_option('precision', 2)

# 載入訓練統計到 DataFrame 中
df_stats = pd.DataFrame(data=training_stats)

# 使用 epoch 值作為每行的索引
df_stats = df_stats.set_index('epoch')

# 展示表格資料
df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,0.17,0.19,0.93,0:07:05,0:01:12
2,0.04,0.15,0.96,0:07:20,0:01:12


In [ ]:
#保存第一次訓練完成的模型
firstmodel=model

In [ ]:
#重置狀態為第一次訓練完成的模型
#後續都從這邊開始第二次訓練

model=firstmodel

In [ ]:
#修正
import pandas as pd

# 載入資料集到 pandas 的 dataframe 中
df0=pd.read_csv("label0.csv")
df0
df1=pd.read_csv("label1.csv")
df1

#shape可以回傳目前dataframe的列數,欄數
#shape[0]回傳列數 shape[1]回傳欄數
#print(df.shape)

df0=df0.sample(frac=1.0)
df1=df1.sample(frac=1.0)

df=df0.iloc[:560, :]
df0=df0.iloc[560:2800, :]
# 列印資料集的記錄數
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

df=df.append(df1.iloc[:560, :],ignore_index=True)
df1=df1.iloc[560:2800, :]
# 列印資料集的記錄數
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

print('Number of df0: {:,}\n'.format(df0.shape[0]))
print('Number of df1: {:,}\n'.format(df1.shape[0]))

Number of training sentences: 560

Number of training sentences: 1,120

Number of df0: 2,240

Number of df1: 2,240



In [ ]:
# 載入 BERT 分詞器
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


# 構建 text_和 labels 列表
text_= df.text_.values
labels = df.label.values #0為CG(寫手評論) 1為OR(原始評論)
print(' Original: ', text_[1])# 輸出原始句子
print('Tokenized: ', tokenizer.tokenize(text_[1]))# 將分詞後的內容輸出
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_[1])))# 將每個詞對映到詞典下標
input_ids = []
attention_masks = []

for sent in text_:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # 輸入文字
                        add_special_tokens = True, # 新增 '[CLS]' 和 '[SEP]'
                        max_length =310,         # 填充 & 截斷長度
                        pad_to_max_length = True,
                        return_attention_mask = True,   # 返回 attn. masks.
                        return_tensors = 'pt',     # 返回 pytorch tensors 格式的資料
                   )

    # 將編碼後的文字加入到列表
    input_ids.append(encoded_dict['input_ids'])

    # 將文字的 attention mask 也加入到 attention_masks 列表
    attention_masks.append(encoded_dict['attention_mask'])

# 將列表轉換為 tensor
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# 輸出第 1 行文字的原始和編碼後的資訊
print('Original: ', text_[0])
print('Token IDs:', input_ids[0])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Loading BERT tokenizer...
 Original:  The mic on my headset doesn't work on this item!So it just don't work properly!Defective item!I should read the other commends before buy this item!
Tokenized:  ['the', 'mic', 'on', 'my', 'heads', '##et', 'doesn', "'", 't', 'work', 'on', 'this', 'item', '!', 'so', 'it', 'just', 'don', "'", 't', 'work', 'properly', '!', 'defective', 'item', '!', 'i', 'should', 'read', 'the', 'other', 'com', '##men', '##ds', 'before', 'buy', 'this', 'item', '!']
Token IDs:  [1996, 23025, 2006, 2026, 4641, 3388, 2987, 1005, 1056, 2147, 2006, 2023, 8875, 999, 2061, 2009, 2074, 2123, 1005, 1056, 2147, 7919, 999, 28829, 8875, 999, 1045, 2323, 3191, 1996, 2060, 4012, 3549, 5104, 2077, 4965, 2023, 8875, 999]


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  This thing is dream. Small wheel radius means easy to pedal. I have the single speed, no hills here in beach.. It is a delight to ride. Moves along just fine. I go to the store, cuts down on gas usage and fighting the other drivers (very impatient drivers here).
Token IDs: tensor([  101,  2023,  2518,  2003,  3959,  1012,  2235,  5217, 12177,  2965,
         3733,  2000, 15749,  1012,  1045,  2031,  1996,  2309,  3177,  1010,
         2053,  4564,  2182,  1999,  3509,  1012,  1012,  2009,  2003,  1037,
        12208,  2000,  4536,  1012,  5829,  2247,  2074,  2986,  1012,  1045,
         2175,  2000,  1996,  3573,  1010,  7659,  2091,  2006,  3806,  8192,
         1998,  3554,  1996,  2060,  6853,  1006,  2200, 17380,  6853,  2182,
         1007,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0

In [ ]:
from torch.utils.data import TensorDataset, random_split

# 將輸入資料合併為 TensorDataset 物件
dataset = TensorDataset(input_ids, attention_masks, labels)

# 計算訓練集和驗證集大小
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# 按照資料大小隨機拆分訓練集和測試集
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

  896 training samples
  224 validation samples


In [ ]:
# 在 fine-tune 的訓練中，BERT 作者建議小批量大小設為 16 或 32
batch_size = 32


# 為訓練和驗證集建立 Dataloader，對訓練樣本隨機洗牌
train_dataloader = DataLoader(
            train_dataset,  # 訓練樣本
            sampler = RandomSampler(train_dataset), # 隨機小批量
            batch_size = batch_size # 以小批量進行訓練
        )

# 驗證集不需要隨機化，這裡順序讀取就好
validation_dataloader = DataLoader(
            val_dataset, # 驗證樣本
            sampler = SequentialSampler(val_dataset), # 順序選取小批量
            batch_size = batch_size
        )



for t_batch in validation_dataloader:
  print(t_batch[0])
  print(t_batch[1])
  print(t_batch[2])

tensor([[  101,  2023, 10412,  ...,     0,     0,     0],
        [  101,  2008,  2015,  ...,     0,     0,     0],
        [  101,  1996,  1004,  ...,     0,     0,     0],
        ...,
        [  101,  1045,  2293,  ...,     0,     0,     0],
        [  101,  1996,  3105,  ...,  1997,  3684,   102],
        [  101,  2023,  2003,  ...,     0,     0,     0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0,
        0, 0, 0, 1, 1, 1, 1, 1])
tensor([[ 101, 2023, 7069,  ...,    0,    0,    0],
        [ 101, 3835, 3737,  ...,    0,    0,    0],
        [ 101, 1045, 2363,  ...,    0,    0,    0],
        ...,
        [ 101, 1045, 2097,  ...,    0,    0,    0],
        [ 101, 2023, 2003,  ...,    0,    0,    0],
        [ 101, 1045, 2734,  ...,  

In [ ]:
# 在 gpu 中執行該模型
model.cuda()

# 我認為 'W' 代表 '權重衰減修復"
optimizer = AdamW(model.parameters(),
                  lr = 7e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                )


# 訓練 epochs。 BERT 作者建議在 2 和 4 之間，設大了容易過擬合
epochs = 2
# 總的訓練樣本數
total_steps = len(train_dataloader) * epochs

# 建立學習率排程器
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)



# 根據預測結果和標籤資料來計算準確率
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    #print("pred_flat",pred_flat)
    labels_flat = labels.flatten()
    #print("labels_flat",labels_flat)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# 設定隨機種子值，以確保輸出是確定的
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 儲存訓練和評估的 loss、準確率、訓練時長等統計指標,
training_stats = []

# 統計整個訓練時長
total_t0 = time.time()

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================


    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 統計單次 epoch 的訓練時間
    t0 = time.time()

    # 重置每次 epoch 的訓練總 loss
    total_train_loss = 0

    # 將模型設定為訓練模式。這裡並不是呼叫訓練介面的意思
    # dropout、batchnorm 層在訓練和測試模式下的表現是不同的 (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # 訓練集小批量迭代
    for step, batch in enumerate(train_dataloader):

        # 每經過40次迭代，就輸出進度資訊
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 準備輸入資料，並將其拷貝到 gpu 中
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # 每次計算梯度前，都需要將梯度清 0，因為 pytorch 的梯度是累加的
        model.zero_grad()

        # 前向傳播
        # 文件參見:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # 該函式會根據不同的引數，會返回不同的值。 本例中, 會返回 loss 和 logits -- 模型的預測結果
        loss, logits = model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels,
                             return_dict =False)

        # 累加 loss
        total_train_loss += loss.item()

        # 反向傳播
        loss.backward()

        # 梯度裁剪，避免出現梯度爆炸情況
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 更新引數
        optimizer.step()

        # 更新學習率
        scheduler.step()

    # 平均訓練誤差
    avg_train_loss = total_train_loss / len(train_dataloader)

    # 單次 epoch 的訓練時長
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    # ========================================
    #               Validation
    # ========================================
    # 完成一次 epoch 訓練後，就對該模型的效能進行驗證

    print("")
    print("Running Validation...")

    t0 = time.time()

    # 設定模型為評估模式
    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        # 將輸入資料載入到 gpu 中
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # 評估的時候不需要更新引數、計算梯度
        with torch.no_grad():
            (loss, logits) = model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels,
                                   return_dict =False)

        # 累加 loss
        total_eval_loss += loss.item()

        # 將預測結果和 labels 載入到 cpu 中計算
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # 計算準確率
        total_eval_accuracy += flat_accuracy(logits, label_ids)


    # 列印本次 epoch 的準確率
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # 統計本次 epoch 的 loss
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    # 統計本次評估的時長
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # 記錄本次 epoch 的所有統計資訊
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 2 ========
Training...

  Average training loss: 0.92
  Training epcoh took: 0:00:46

Running Validation...
  Accuracy: 0.63
  Validation Loss: 0.65
  Validation took: 0:00:04

======== Epoch 2 / 2 ========
Training...

  Average training loss: 0.59
  Training epcoh took: 0:00:46

Running Validation...
  Accuracy: 0.62
  Validation Loss: 0.66
  Validation took: 0:00:04

Training complete!
Total training took 0:01:40 (h:mm:ss)


In [ ]:
import pandas as pd

# 保留 2 位小數
pd.set_option('precision', 2)

# 載入訓練統計到 DataFrame 中
df_stats = pd.DataFrame(data=training_stats)

# 使用 epoch 值作為每行的索引
df_stats = df_stats.set_index('epoch')

# 展示表格資料
df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,0.92,0.65,0.63,0:00:46,0:00:04
2,0.59,0.66,0.62,0:00:46,0:00:04


In [ ]:
secondmodel=model

In [ ]:
model=secondmodel

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving amazon(預測).csv to amazon(預測).csv


In [ ]:
# 根據預測結果和標籤資料來計算準確率
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    print("pred_flat",pred_flat)
    labels_flat = labels.flatten()
    print("labels_flat",labels_flat)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import pandas as pd


# 載入資料集
#df = pd.read_csv("amazon(預測).csv")
df=df0
df=df.append(df1,ignore_index=True)

#測試用程式碼
#df=df[df.LABEL==0]

# 列印資料集大小
print('Number of test sentences: {:,}\n'.format(df.shape[0]))

df.sample(10)
# 將資料集轉換為列表
text_ = df.text_.values
labels = df.label.values

print(text_[0])

# 分詞、填充或截斷
input_ids = []
attention_masks = []
for sent in text_:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        max_length =310,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])


input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

batch_size = 32

# 準備好資料集
prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Number of test sentences: 4,480

Finally opened the box 2 months after purchase. Camera died within minutes after setup. Contacted foscam and they have not yet responded for days already.


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
 # 預測測試集

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))
# 依然是評估模式
model.eval()

# Tracking variables
predictions , true_labels = [], []
total_test_accuracy=0
iii=1

# 預測
for batch in prediction_dataloader:
  # 將資料載入到 gpu 中
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_labels = batch

  # 不需要計算梯度
  with torch.no_grad():
      # 前向傳播，獲取預測結果
      outputs = model(b_input_ids, token_type_ids=None,
                      attention_mask=b_input_mask)

  logits = outputs[0]


  # 將結果載入到 cpu 中
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()




  this_batch_accuracy = flat_accuracy(logits, label_ids)
  total_test_accuracy += this_batch_accuracy
  print(iii,this_batch_accuracy)
  iii+=1



  # 儲存預測結果和 labels
  predictions.append(logits)
  true_labels.append(label_ids)

total_test_accuracy/=(iii-1)
print("total_test_accuracy",total_test_accuracy)
#print('    DONE.')

Predicting labels for 4,480 test sentences...
pred_flat [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 1 0 0 0 0 0 0 0 0 1]
labels_flat [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
1 0.71875
pred_flat [0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 1 0 1 0]
labels_flat [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2 0.75
pred_flat [0 0 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0 0 1 0 1 1]
labels_flat [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
3 0.625
pred_flat [0 0 0 0 1 0 1 0 1 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0]
labels_flat [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
4 0.71875
pred_flat [1 1 0 0 1 0 0 0 1 1 0 1 0 1 1 0 1 0 0 0 0 0 1 0 1 0 1 1 0 1 0 0]
labels_flat [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
5 0.5625
pred_flat [0 1 0 1 1 1 1 0 0 0 1 0 1 1 0 1 0 0 1 1 0 0 1 0 1 0 0 0 0 0 0 1]
labels_flat [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
print('Positive samples: %d of %d (%.2f%%)' % (df.label.sum(), len(df.label), (df.label.sum() / len(df.label) * 100.0)))
#print(predictions[0])

#labels=labels.tolist()
print(labels)
print(len(labels))
#print(type(true_labels[0]))


#175個list，每個list中有32個 32*175=5600
print(len(true_labels[87]))
after_true_labels=[]

for i_true_labels in true_labels:
  after_true_labels.extend(i_true_labels.tolist())

print(after_true_labels)
print(len(after_true_labels))

Positive samples: 2240 of 4480 (50.00%)
tensor([0, 0, 0,  ..., 1, 1, 1])
4480
32
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
import array
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
y_pred=after_true_labels
y_true=labels

print(y_pred)

print(confusion_matrix(y_pred,y_true,labels=[0,1]))


tn, fp, fn, tp =confusion_matrix(y_pred,y_true,labels=[0,1]).ravel()
print(tn, fp, fn, tp)

print('accuracy_score : ', accuracy_score(y_pred,y_true))
print('precision_score : ', precision_score(y_pred,y_true))
print('recall_score : ', recall_score(y_pred,y_true))
print('f1_score : ', f1_score(y_pred,y_true))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
from sklearn.metrics import matthews_corrcoef

matthews_set = []

# 計算每個 batch 的 MCC
print('Calculating Matthews Corr. Coef. for each batch...')

# For each input batch...
for i in range(len(true_labels)):
  pred_labels_i = np.argmax(predictions[i], axis=1).flatten()

  # 計算該 batch 的 MCC
  matthews = matthews_corrcoef(true_labels[i], pred_labels_i)
  matthews_set.append(matthews)

Calculating Matthews Corr. Coef. for each batch...


In [ ]:
# 合併所有 batch 的預測結果
flat_predictions = np.concatenate(predictions, axis=0)

# 取每個樣本的最大值作為預測值
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# 合併所有的 labels
flat_true_labels = np.concatenate(true_labels, axis=0)

# 計算 MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

print('Total MCC: %.3f' % mcc)

Total MCC: 0.270
